<a href="https://colab.research.google.com/github/bignamu/python-AI/blob/main/torch_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import numpy as np
import sys


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(1, 3)
        self.fc2 = nn.Linear(3, 1)
        self.batch_norm1 = nn.BatchNorm1d(10) # 노드의 개수?
        self.relu = nn.ReLU(inplace=True)
        # self.dropout = nn.Dropout(0.7)

    def forward(self, x):
        print(x.shape)
        print(x)
        x = self.fc1(x)
        print(x.shape)
        print(x)
        # x = self.batch_norm1(x)
        # x = self.relu(x)
        # # x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        print(x.shape)
        print(x)

        return x

In [ ]:

class CustomDataset(Dataset):
  def __init__(self, data, transforms=None):
      self.x = [i[0] for i in data]
      self.y = [i[1] for i in data]


  def __len__(self):
    return len(self.x)


  def __getitem__(self, idx):
      x = [self.x[idx]]
      y = self.y[idx]
      x= np.array(x)


      return x, y



In [ ]:

torch.manual_seed(1)
data = [[2,0], [4,0], [6,0], [8,1], [10,1], [12,1]]

train_dataset = CustomDataset(data, transforms=None)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = Net().to(device)

for x, y in train_loader:

    x = x.float().to(device)
    y = y.float().to(device)
    print(x)
    print('------------------'*5)
    outputs = model(x)
    print('------------------'*5)
    print(outputs)
    outputs = outputs.detach().numpy()
    print(outputs)
    print(y)
    break

tensor([[2.]])
------------------------------------------------------------------------------------------
torch.Size([1, 1])
tensor([[2.]])
torch.Size([1, 3])
tensor([[ 1.4999, -1.8242,  0.2120]], grad_fn=<AddmmBackward>)
torch.Size([1, 1])
tensor([[0.]], grad_fn=<ReluBackward1>)
------------------------------------------------------------------------------------------
tensor([[0.]], grad_fn=<ReluBackward1>)
[[0.]]
tensor([0.])


# Pytorch Class 구조

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(1, 1)
        self.fc2 = nn.Linear(3, 1)
        self.batch_norm1 = nn.BatchNorm1d(3)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.7)
        self.sig = nn.Sigmoid() # 활성화 함수

    def forward(self, x):
        # print(x.shape)
        x = self.fc1(x)
        # x = self.batch_norm1(x)
        # x = self.relu(x)
        # x = self.dropout(x)
        # x = self.fc2(x)
        # x = self.relu(x)
        # x = self.sig(x)
         #H(X)식에 입력X 로부터 예측된 y를 얻는 것을 forward 연산이라고 함
        return x


class CustomDataset(Dataset):
    def __init__(self, data, transforms=None):
        self.x = [i[0] for i in data]
        self.y = [i[1] for i in data]
         # 데이터셋의 전처리를 해주는 부분


    def __len__(self):
        return len(self.x)
        # 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분

    def __getitem__(self, idx):
        x = [self.x[idx]]
        y = self.y[idx]
        x = np.array(x)
        y = np.array(y)
        #  데이터셋에서 특정 샘플을 가져오는 함수 ? forward

        return x, y

In [ ]:
!cd /content/drive/MyDrive/Colab Notebooks/Pytorch
!pwd
!ls

/bin/bash: line 0: cd: too many arguments
/content
drive  sample_data


In [ ]:
data = [[2, 4], [4, 8], [6, 12], [8, 16], [10, 20], [12, 24]]

torch.manual_seed(1)

# 커스텀 데이터 로드 과정
train_dataset = CustomDataset(data, transforms=None)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True)
# 미니 배치 학습을 하게되면 미니 배치만큼만 가져가서 미니 배치에 대한 대한 비용(cost)를 계산하고, gradient descent을 수행합니다. 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = Net().to(device)
criterion = nn.MSELoss() # 비용함수 선언 MSE의 최솟값이 훈련 데이터를 가장 잘 나타내는 직선을 나타낼것
# optimizer = optim.Adam(model.parameters(), lr=0.01)


optimizer =torch.optim.SGD(model.parameters() , lr=  0.001) 
# 최소의 W b를 찾아내는 과정 optimizer를 SGD를 사용했다는 뜻
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.1) 
#일정한 Step 마다 learning rate에 gamma를 곱해주는 방식
epoch = 1000
total_loss = 0
for i in range(epoch):
    for x, y in train_loader:
        x = x.float().to(device)
        y = y.float().to(device)
        # print('x,y', x,y)
        # print(x, y)
        outputs = model(x)
        # print('outputs ',outputs)
        loss = criterion(outputs, y)
        optimizer.zero_grad()  # 기울기 초기화 //가중치와 편향의 초기화
        loss.backward()  # 가중치와 편향에 대해 기울기 계산
        total_loss += loss.item() # 텐서안에 있는 값을 나타냄
        outputs = outputs.detach().numpy()
        # print('outputs ',outputs)
        y = y.numpy()
        # sys.exit()
        # print(outputs)
        # print(y)
    if i == 999:
        torch.save(model.state_dict(), f'weight/model_last.pth')
    print(f"epoch -> {i}      loss -- > ", total_loss / len(train_loader) if epoch i % 10 == 0)
    optimizer.step()
    total_loss = 0


SyntaxError: ignored

In [ ]:

model.eval() # W b 고정하겠다
model.load_state_dict(torch.load('weight/model_last.pth'))
test_data = [[1, 2],[3, 6],[5, 10]]
test_dataset = CustomDataset(test_data, transforms=None)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
for x, y in test_loader:
    x = x.float().to(device)
    y = y.long().to(device)
    # print('x, y : ', x, y)
    outputs = model(x)
    print('x, y, outputs : ',x,y,outputs)

model.train()

x, y, outputs :  tensor([[1.]]) tensor([2]) tensor([[3.8605]], grad_fn=<AddmmBackward>)
x, y, outputs :  tensor([[3.]]) tensor([6]) tensor([[6.6733]], grad_fn=<AddmmBackward>)
x, y, outputs :  tensor([[5.]]) tensor([10]) tensor([[9.4862]], grad_fn=<AddmmBackward>)


Net(
  (fc1): Linear(in_features=1, out_features=1, bias=True)
  (fc2): Linear(in_features=3, out_features=1, bias=True)
  (batch_norm1): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (dropout): Dropout(p=0.7, inplace=False)
  (sig): Sigmoid()
)

# Pytorch Iris

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import torch.nn.functional as F
import torch.optim

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(4, 3)
        # self.fc2 = nn.Linear(3, 1)
        # self.batch_norm1 = nn.BatchNorm1d(3)
        # self.relu = nn.ReLU(inplace=True)
        self.sig = nn.Sigmoid() # 활성화 함수

    def forward(self, x):
        # print(x.shape)
        x = self.fc1(x)
        # x = self.batch_norm1(x)
        # x = self.relu(x)
        # x = self.dropout(x)
        # x = self.fc2(x)
        # x = self.relu(x)
        x = self.sig(x) * x
        #H(X)식에 입력X 로부터 예측된 y를 얻는 것을 forward 연산이라고 함
        return x


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X,y, transforms=None):
        self.x = [i for i in X]
        self.y = [i for i in y]
         # 데이터셋의 전처리를 해주는 부분


    def __len__(self):
        return len(self.x)
        # 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        x = np.array(x)
        y = np.array(y)

        #  데이터셋에서 특정 샘플을 가져오는 함수 ? forward

        return x, y

In [ ]:
iris_dataset = load_iris()

In [ ]:
print(iris_dataset.keys(), type(iris_dataset))

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename']) <class 'sklearn.utils.Bunch'>


In [ ]:
print(iris_dataset['data'].shape, iris_dataset['target_names'].shape)

(150, 4) (3,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                                    iris_dataset['data'],
                                                    iris_dataset['target'],
                                                    stratify=iris_dataset['target'],
                                                    random_state=0
                                                    )
# stratify 분류 라벨 샘플 분산시키기

In [ ]:
torch.manual_seed(1)

train_dataset = CustomDataset(X_train,y_train, transforms=None)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = Net().to(device)
criterion = nn.CrossEntropyLoss() # 비용함수
optimizer = torch.optim.Adam(model.parameters(),lr=0.01) # gradient descent

In [ ]:
epoch = 100
total_loss = 0
print(len(train_loader))
print(train_loader.dataset.y[:5])

112
[1, 0, 0, 1, 2]


In [ ]:
for i in range(epoch):
  total_loss = 0
  #print(i)
  for X_train, y_train in train_loader:
    #print(X_train,y_train)
    X_train = X_train.float().to(device) # 차원 분별필요
    y_train = y_train.long().to(device) #long

    outputs = model(X_train)
    #print(outputs,X_train,y_train)
    loss = criterion(outputs, y_train) # outputs.view_as...
    optimizer.zero_grad()  # 기울기 초기화 //가중치와 편향의 초기화
    loss.backward()  # 가중치와 편향에 대해 기울기 계산
    total_loss += loss.item() # 텐서안에 있는 값을 나타냄
    outputs = outputs.detach().numpy()
    # print('outputs ',outputs)
    y_train = y_train.numpy()
    optimizer.step()
    total_loss = 0


In [ ]:

model.eval() # W b 고정하겠다
# model.load_state_dict(torch.load('weight/model_last.pth'))
test_dataset = CustomDataset(X_test,y_test, transforms=None)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
for x, y in test_loader:
    x = x.float().to(device)
    y = y.long().to(device)
    # print('x, y : ', x, y)
    outputs = model(x)
    _, predicted = torch.max(outputs,0)
    #print('x, y, outputs : ',x,y,predicted)
